In [1]:
import pandas as pd
import numpy as np
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from collections import Counter

from itertools import chain

In [2]:
# nltk.download()

In [3]:
violations_section = pd.read_csv("../data/violations_section.csv")
violations_TCA = pd.read_csv("../data/violations_TCA.csv")
violations_MCL = pd.read_csv("../data/violations_MCL.csv")
violations_description_id = pd.read_csv("../data/violations_description_id.csv")
violations_remainder = pd.read_csv("../data/violations_remainder.csv")

In [4]:
violations_list = [violations_section, violations_TCA, violations_MCL, violations_description_id, violations_remainder]

In [5]:
for violation in violations_list:
    assert 'Unnamed: 0' in violation.columns

In [6]:
for violation in violations_list:
    violation.set_index('Unnamed: 0', inplace=True)
    violation.index.rename('Original_Index', inplace=True)

In [7]:
violations_description_id.head()

,id,type,desc,text,remedial
Original_Index,,,,,
15,(B). It is unlawful for any person to dispose ...,NaN,N,NaN,1.0
438,he or she shall serve notice of the risk and t...,NaN,NaN,NaN,NaN
439,tree at the expense of the property owner,NaN,N,NaN,2.0
707,"human occupation or use, upon failure or refus...",NaN,NaN,NaN,NaN
708,"remove or demolish, such dwelling or structure...",NaN,N,NaN,2.0


In [8]:
violations_description_id['text'] = violations_description_id['id'].values

In [9]:
violations_description_id.head()

,id,type,desc,text,remedial
Original_Index,,,,,
15,(B). It is unlawful for any person to dispose ...,NaN,N,(B). It is unlawful for any person to dispose ...,1.0
438,he or she shall serve notice of the risk and t...,NaN,NaN,he or she shall serve notice of the risk and t...,NaN
439,tree at the expense of the property owner,NaN,N,tree at the expense of the property owner,2.0
707,"human occupation or use, upon failure or refus...",NaN,NaN,"human occupation or use, upon failure or refus...",NaN
708,"remove or demolish, such dwelling or structure...",NaN,N,"remove or demolish, such dwelling or structure...",2.0


In [10]:
for violation in violations_list:
    violation.drop('id', axis='columns', inplace=True)
    
    violation['desc'].replace(np.nan, '', regex=True, inplace=True)
    violation['text'].replace(np.nan, '', regex=True, inplace=True)

In [11]:
violations_section.head()

,type,desc,text,remedial
Original_Index,,,,
0,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS
1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER
2,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT
3,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT
4,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE


In [12]:
violations_TCA.head()

,type,desc,text,remedial
Original_Index,,,,
106,LANDREG,Landlord Registration Required,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,CAAR_LAND_REG
376,CAAR_LAND_REG,LANDLORD REGISTRATION REQUIRED,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,NaN
807,BT575101A2,Not Manufacturer and Wholesale Retail,T.C.A 57-5-101(a)(2): Except as otherwise prov...,NaN
808,BT575101A3,Not Wholesale and Manufacturer Retail,T.C.A 57-5-101(a)(3): Except as otherwise prov...,NaN
809,BT575103A1,Operating without county city permit,T.C.A 57-5-103(a)(1): It is unlawful to operat...,NaN


In [13]:
violations_MCL.head()

,type,desc,text,remedial
Original_Index,,,,
481,CAGA_DEX,DOG EXCREMENT,MCL Chapter 8.04.180 – Removal of Excrement. ...,NaN
482,CAGB_HAZ_W,HAZARDOUS WASTE,"MCL Chapter 10.20.150 - Hazardous, pathogenic ...",NaN
483,CAGC_GCL,GARBAGE CONTAINER LIDS,MCL Chapter 10.20.160 – Container Requirements...,NaN
484,CAGD_ID,ILLEGAL DUMPING,MCL Chapter 10.20.320 – Illegal Dumping. It i...,NaN
485,CAGE_LPR,LITTER ON PRIVATE PROPERTY,MCL Chapter 10.24.070 - Litter on private prop...,NaN


In [14]:
violations_description_id.head()

,type,desc,text,remedial
Original_Index,,,,
15,NaN,N,(B). It is unlawful for any person to dispose ...,1.0
438,NaN,,he or she shall serve notice of the risk and t...,NaN
439,NaN,N,tree at the expense of the property owner,2.0
707,NaN,,"human occupation or use, upon failure or refus...",NaN
708,NaN,N,"remove or demolish, such dwelling or structure...",2.0


In [15]:
violations_remainder.head()

,type,desc,text,remedial
Original_Index,,,,
126,MVENVEND,Mobile Vending,Ordinance BL2006-1283 - Mobile Vendor: The ped...,CAAZ_MVEN_VENDING
196,TRASHCANS,Trash Cans,Ordinance 89-826 - Trash Cans: Refuse containe...,CAAH_TRASH_CANS
209,SWVIOLPTP,Violations - Post Construction Treatment,See SW Code,NaN
210,SWVIOLWQ,Violations for Water Quality Impacts,See SW Code,NaN
211,SWENVCRT,Environmental Court Proceedings,See SW Code,NaN


In [16]:
master_violations = pd.concat(violations_list)

In [17]:
master_violations

,type,desc,text,remedial
Original_Index,,,,
0,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS
1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER
2,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT
3,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT
4,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE
5,BOARDINGRQ,Boarding Requirements,Section 16.24.340 (U) (2) – Boarding of Vacant...,CAAH_BOARDING_REQU
6,BOARDVAC,Boarding of Vacant Buildings,Section 16.24.340 (U) (1) – Boarding of Vacant...,CAAH_BOARDING_VABLDG
7,CARROW,Vehicle In The Right-Of-Way,Section 12.08.210 - Abandoned Vehicles: Abando...,CAMCL_SEW_CONCT
8,CERTCOMP,Certificate of Compliance Required,Section 17.40.580 - Certificate of Compliance:...,CAAZ_CERT_COMP


In [18]:
def tokenize_column(column_values):
    return word_tokenize(column_values)
    
def tokenize_row(index, row):
    desc = row['desc']
    text = row['text']
    
    desc_tokenized, text_tokenized = [], []
    
    if desc:
        desc_tokenized = tokenize_column(desc)
        
    if text:
        text_tokenized = tokenize_column(text)

    return (index, desc_tokenized, text_tokenized)

In [19]:
tokenized_data = list()

for index, row in master_violations.iterrows():
    tokenized_row_data = tokenize_row(index, row)
    tokenized_data.append(tokenized_row_data)

In [20]:
tokenized_df = pd.DataFrame(tokenized_data, columns=['Original_Index', 'tokenized_desc', 'tokenized_text'])
tokenized_df.set_index('Original_Index', inplace=True)

tokenized_df.head()

,tokenized_desc,tokenized_text
Original_Index,,
0,[Banners],"[Section, 17.32.060, &, 17.32.070, -, Banners,..."
1,"[Proper, Maintenance, Req, -, Owner]","[Section, 16.16.030, (, B, ), -, Proper, maint..."
2,"[Proper, Maintenance, Req]","[Section, 16.16.030, (, A, ), -, Proper, maint..."
3,"[Building, Permit, Required]","[Section, 16.28.010, -, Building, Permit, Requ..."
4,"[Scope, of, Building, Code]","[Section, 16.16.020, (, B, ), -, The, provisio..."


In [21]:
master_violations = master_violations.join(tokenized_df)
master_violations.head()

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
0,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS,[Banners],"[Section, 17.32.060, &, 17.32.070, -, Banners,..."
1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER,"[Proper, Maintenance, Req, -, Owner]","[Section, 16.16.030, (, B, ), -, Proper, maint..."
2,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT,"[Proper, Maintenance, Req]","[Section, 16.16.030, (, A, ), -, Proper, maint..."
3,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT,"[Building, Permit, Required]","[Section, 16.28.010, -, Building, Permit, Requ..."
4,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE,"[Scope, of, Building, Code]","[Section, 16.16.020, (, B, ), -, The, provisio..."


In [22]:
#subset that contains rows with the regular expression "Section #." in text column
master_section = master_violations[master_violations.text.str.match(r"(S|s)ection\s(\d+)\.", na=False)]

#new df where rows containing "Section #" are removed
master_nosection = master_violations.drop(master_section.index)

#subset that contains rows with variations of TCA in text column
master_TCA = master_nosection[master_nosection.text.str.match("(T.C.A|Tennessee Code Annotated)", na=False)]

#new df where rows containing "T.C.A" or Tennessee Code Annotated" are removed
master_noTCA = master_nosection.drop(master_TCA.index)

#subset that contains variations of MCL in text column 
#Be aware MCL and MCL Chapter appear to be different types of codes
master_MCL = master_noTCA[master_noTCA.text.str.match("(M.C.L|MCL Chapter)", na=False)]

#new df where rows containing "M.C.L" or "MCL Chapter" are removed
master_remainder = master_noTCA.drop(master_MCL.index)
#master_remainder

In [23]:
#export csv files containing subsets for section, TCA, MCL, descriptions in id column, and remaining entries.
master_section.to_csv("../data/master_section.csv")
master_TCA.to_csv("../data/master_TCA.csv")
master_MCL.to_csv("../data/master_MCL.csv")
master_remainder.to_csv("../data/master_remainder.csv")

In [24]:
master_MCL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 481 to 881
Data columns (total 6 columns):
type              101 non-null object
desc              101 non-null object
text              101 non-null object
remedial          0 non-null object
tokenized_desc    101 non-null object
tokenized_text    101 non-null object
dtypes: object(6)
memory usage: 5.5+ KB


In [25]:
MCL_MCL = master_MCL[master_MCL.text.str.match(r"M.C.L")]
MCL_MCL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 735 to 881
Data columns (total 6 columns):
type              76 non-null object
desc              76 non-null object
text              76 non-null object
remedial          0 non-null object
tokenized_desc    76 non-null object
tokenized_text    76 non-null object
dtypes: object(6)
memory usage: 4.2+ KB


In [26]:
MCL_Chapter = master_MCL[master_MCL.text.str.match(r"MCL Chapter")]

In [27]:
MCL_list = MCL_MCL["tokenized_desc"].tolist()
MCL_flat_list = [item for sublist in MCL_list for item in sublist]
#MCL_set = set(MCL_flat_list)
#MCL_set

MCL_kw_freq = dict(Counter(MCL_flat_list))
MCL_kw_freq

MCL_kw_s = pd.Series(MCL_kw_freq, name="freq")

MCL_kw = pd.DataFrame(data = MCL_kw_s, columns = ["freq"])
MCL_kw = MCL_kw.reset_index()
MCL_kw.columns = ["kw", "freq"]
#MCL_kw.head()
MCL_kw_sorted = MCL_kw.sort_values(by = "freq", ascending = False)
MCL_kw_sorted.head()

#results = []
#for tokenlist in MCL_list:
#    sentence_results = []
#    for token in tokenlist:
#        sentence_results.append(word_tokenize(token))
#    results.append(sentence_results)

#Counter(chain(MCL_list))

,kw,freq
12,BB,51
127,permit,9
119,on,8
0,",",6
153,sales,6


In [44]:
searchfor = [
    'BB ', 'sale', 'caterer', 'beer', 'moral', 'application', 
    "dist", "patron", "attire", "privilege", "suspension", "reproductions"]
MCL_drop = MCL_MCL.drop(MCL_MCL[MCL_MCL["desc"].str.contains('|'.join(searchfor), flags=re.IGNORECASE)].index)

#s[s.str.contains('|'.join(searchfor))]

#MCL_noBB = MCL_MCL.drop(MCL_MCL[MCL_MCL["desc"].str.contains("BB ")].index)


In [45]:
MCL_MCL

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
735,BB708020,Sales without a permit,M.C.L 7.08.020: No person shall sell beer with...,NaN,"[Sales, without, a, permit]","[M.C.L, 7.08.020, :, No, person, shall, sell, ..."
736,BB708030B,Off Sale - consume on or near premises,"M.C.L 7.08.030B: A retailer's ""off-sale"" permi...",NaN,"[Off, Sale, -, consume, on, or, near, premises]","[M.C.L, 7.08.030B, :, A, retailer, 's, ``, off..."
737,BB708030C,On sale required for on premise consume,"M.C.L 7.08.030C: A retailer's ""on-sale"" permit...",NaN,"[On, sale, required, for, on, premise, consume]","[M.C.L, 7.08.030C, :, A, retailer, 's, ``, on-..."
738,BB708030D,Caterer Permit,M.C.L 7.08.030D: A caterer's permit shall be i...,NaN,"[Caterer, Permit]","[M.C.L, 7.08.030D, :, A, caterer, 's, permit, ..."
739,BB708030E,Special Event - sales on premises,M.C.L 7.08.030E: A retailer's special events p...,NaN,"[Special, Event, -, sales, on, premises]","[M.C.L, 7.08.030E, :, A, retailer, 's, special..."
740,BB708040B3,Owners convict beer laws moral turpitude,"M.C.L 7.08.040B3: That no person, firm, corpor...",NaN,"[Owners, convict, beer, laws, moral, turpitude]","[M.C.L, 7.08.040B3, :, That, no, person, ,, fi..."
741,BB708040B5,Owners incarcerated moral turpitude,M.C.L 7.08.040B5: That no person having at lea...,NaN,"[Owners, incarcerated, moral, turpitude]","[M.C.L, 7.08.040B5, :, That, no, person, havin..."
742,BB708040G,Supplemental Application Information,M.C.L 7.08.040G: An applicant or permit holder...,NaN,"[Supplemental, Application, Information]","[M.C.L, 7.08.040G, :, An, applicant, or, permi..."
743,BB708040H,False statement on application,M.C.L 7.08.040H: Any applicant making a false ...,NaN,"[False, statement, on, application]","[M.C.L, 7.08.040H, :, Any, applicant, making, ..."


In [48]:
Chapter_list = MCL_Chapter["tokenized_desc"].tolist()
Chapter_flat_list = [item for sublist in Chapter_list for item in sublist]


Chapter_kw_freq = dict(Counter(Chapter_flat_list))
#Chapter_kw_freq

Chapter_kw_s = pd.Series(Chapter_kw_freq, name="freq")

Chapter_kw = pd.DataFrame(data = Chapter_kw_s, columns = ["freq"])
Chapter_kw = Chapter_kw.reset_index()
Chapter_kw.columns = ["kw", "freq"]
#MCL_kw.head()
Chapter_kw_sorted = Chapter_kw.sort_values(by = "freq", ascending = False)
Chapter_kw_sorted.head()

,kw,freq
19,GARBAGE,5
39,SEWAGE,4
36,RAT,4
21,HARBORAGE,4
0,",",3


In [47]:
searchfor_2 = [
    "garbage", "sew", "vermin", "water", "plumbing", "excrement", 
    "litter", "junk", "waste", "dumping", "vegetation", "chickens"]
Chapter_drop = MCL_Chapter.drop(
    MCL_Chapter[MCL_Chapter["desc"].str.contains('|'.join(searchfor_2), flags=re.IGNORECASE)].index)
MCL_Chapter

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
481,CAGA_DEX,DOG EXCREMENT,MCL Chapter 8.04.180 – Removal of Excrement. ...,NaN,"[DOG, EXCREMENT]","[MCL, Chapter, 8.04.180, –, Removal, of, Excre..."
482,CAGB_HAZ_W,HAZARDOUS WASTE,"MCL Chapter 10.20.150 - Hazardous, pathogenic ...",NaN,"[HAZARDOUS, WASTE]","[MCL, Chapter, 10.20.150, -, Hazardous, ,, pat..."
483,CAGC_GCL,GARBAGE CONTAINER LIDS,MCL Chapter 10.20.160 – Container Requirements...,NaN,"[GARBAGE, CONTAINER, LIDS]","[MCL, Chapter, 10.20.160, –, Container, Requir..."
484,CAGD_ID,ILLEGAL DUMPING,MCL Chapter 10.20.320 – Illegal Dumping. It i...,NaN,"[ILLEGAL, DUMPING]","[MCL, Chapter, 10.20.320, –, Illegal, Dumping,..."
485,CAGE_LPR,LITTER ON PRIVATE PROPERTY,MCL Chapter 10.24.070 - Litter on private prop...,NaN,"[LITTER, ON, PRIVATE, PROPERTY]","[MCL, Chapter, 10.24.070, -, Litter, on, priva..."
486,CAGF_LPU,LITTER ON PUBLIC PROPERTY,MCL Chapter 10.24.140 - Litter on vacant lots....,NaN,"[LITTER, ON, PUBLIC, PROPERTY]","[MCL, Chapter, 10.24.140, -, Litter, on, vacan..."
487,CAGG_LVH,LITTER FROM VEHICLES,MCL Chapter 10.24.150 - Litter from vehicles--...,NaN,"[LITTER, FROM, VEHICLES]","[MCL, Chapter, 10.24.150, -, Litter, from, veh..."
488,CAGH_HWG,EXCESS VEGETATION,MCL Chapter 10.26.010 – Every premise includin...,NaN,"[EXCESS, VEGETATION]","[MCL, Chapter, 10.26.010, –, Every, premise, i..."
489,CAGI_GCAN,GARBAGE APPROVED CONTAINERS,MCL Chapter 10.32.120 - All garbage and trash ...,NaN,"[GARBAGE, APPROVED, CONTAINERS]","[MCL, Chapter, 10.32.120, -, All, garbage, and..."
